In [30]:
import numpy as np
from numba import cuda
import numba as nb

In [64]:
@nb.vectorize(['int32(int32)'], target='cuda')
def check_prime_gpu(num):
    for i in range(2, (num // 2) + 1):
       if (num % i) == 0:
           return 0
    return num

In [146]:
@cuda.jit
def is_prime(n, result):
    index = cuda.grid(1)
    if index < n.size:
        num = n[index]
        if num < 2:
            result[index] = False
        else:
            result[index] = True
            for i in range(2, int(num**0.5) + 1):
                if num % i == 0:
                    result[index] = False
                    break

def find_primes(n):
    result = np.zeros_like(n, dtype=bool)
    threads_per_block = 1024
    blocks_per_grid = (n.size + (threads_per_block - 1)) // threads_per_block
    is_prime[blocks_per_grid, threads_per_block](n, result)
    print(blocks_per_grid, threads_per_block)
    print(result)
    return n[result]

In [147]:
numbers = np.arange(1, 100000, dtype=np.int32)

In [148]:
find_primes(numbers)

98 1024
[False  True  True ... False False False]


c:\Users\omtg\AppData\Local\Programs\Python\Python311\Lib\site-packages\numba\cuda\dispatcher.py:538: NumbaPerformanceWarning: Grid size 98 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\omtg\AppData\Local\Programs\Python\Python311\Lib\site-packages\numba\cuda\cudadrv\devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


array([    2,     3,     5, ..., 99971, 99989, 99991])